In [1]:
import jittor as jt
import numpy as np
import random
from math import sqrt
#jt.flags.use_cuda = True

[SYNC][i 0104 22:44:36.421676 00 compiler.py:847] Jittor(1.2.2.13) src: /home/user3/.local/lib/python3.7/site-packages/jittor
[SYNC][i 0104 22:44:36.424304 00 compiler.py:848] cache_path: /home/user3/.cache/jittor/default/g++
[SYNC][i 0104 22:44:36.438858 00 __init__.py:250] Found /usr/local/cuda/bin/nvcc(10.2.89) at /usr/local/cuda/bin/nvcc.
[SYNC][i 0104 22:44:36.525457 00 __init__.py:250] Found gdb(8.1.0) at /usr/bin/gdb.
[SYNC][i 0104 22:44:36.542424 00 __init__.py:250] Found addr2line(2.30) at /usr/bin/addr2line.
[SYNC][i 0104 22:44:36.589207 00 compiler.py:897] pybind_include: -I/usr/include/python3.7m -I/usr/local/lib/python3.7/dist-packages/pybind11/include


[i 0104 22:44:36.410096 48 __init__.py:250] Found g++(7.5.0) at /usr/bin/g++.


[SYNC][i 0104 22:44:36.613054 00 compiler.py:899] extension_suffix: .cpython-37m-x86_64-linux-gnu.so
[SYNC][i 0104 22:44:36.822905 00 __init__.py:169] Total mem: 62.79GB, using 16 procs for compiling.
[SYNC][i 0104 22:44:36.985427 00 jit_compiler.cc:21] Load cc_path: /usr/bin/g++
[SYNC][i 0104 22:44:37.224330 00 init.cc:52] Found cuda archs: [75,]
[SYNC][i 0104 22:44:37.393473 00 __init__.py:250] Found mpicc(2.1.1) at /usr/bin/mpicc.
[SYNC][i 0104 22:44:37.448424 00 compiler.py:654] handle pyjt_include/home/user3/.local/lib/python3.7/site-packages/jittor/extern/mpi/inc/mpi_warper.h
[SYNC][i 0104 22:44:37.504541 00 compile_extern.py:275] Downloading nccl...
[SYNC][i 0104 22:44:37.586432 00 compile_extern.py:16] found /usr/local/cuda/include/cublas.h
[SYNC][i 0104 22:44:37.589258 00 compile_extern.py:16] found /usr/lib/x86_64-linux-gnu/libcublas.so
[SYNC][i 0104 22:44:38.612463 00 compile_extern.py:16] found /usr/local/cuda/include/cudnn.h
[SYNC][i 0104 22:44:38.615407 00 compile_extern.

## Utility

### Equal Learning Rate

In [2]:
class EqualLR:
    def __init__(self, name):
        self.name = name

    def compute_weight(self, module):
        weight = getattr(module, self.name + '_orig')
        fan_in = weight[0].numel()

        return weight * sqrt(2 / fan_in)

    @staticmethod
    def apply(module, name):
        fn = EqualLR(name)

        weight = getattr(module, name)
        delattr(module, name)
        setattr(module, name + '_orig', weight)
        module.register_pre_forward_hook(fn)

        return fn

    def __call__(self, module, input):
        weight = self.compute_weight(module)
        setattr(module, self.name, weight)


def equal_lr(module, name='weight'):
    EqualLR.apply(module, name)

    return module

class EqualConv2d(jt.Module):
    def __init__(self, *args, **kwargs):
        conv = jt.nn.Conv2d(*args, **kwargs)
        jt.init.gauss_(conv.weight, 0, 1)
        jt.init.constant_(conv.bias,0)
        
        self.conv = equal_lr(conv)

    def execute(self, input):
        return self.conv(input)

class EqualLinear(jt.Module):
    def __init__(self, in_dim, out_dim):
        linear = jt.nn.Linear(in_dim, out_dim)
        jt.init.gauss_(linear.weight, 0, 1)
        jt.init.constant_(linear.bias, 0)

        self.linear = equal_lr(linear)

    def execute(self, input):
        return self.linear(input)

### Blur

In [3]:
class BlurFunctionBackward(jt.Function):
    def execute(self, grad_output, kernel, kernel_flip):
        self.saved_tensors = kernel, kernel_flip

        grad_input = jt.nn.conv2d(
            grad_output, kernel_flip, padding=1, groups=grad_output.shape[1]
        )

        return grad_input

    def grad(self, gradgrad_output):
        kernel, kernel_flip = self.saved_tensors

        grad_input = jt.nn.conv2d(
            gradgrad_output, kernel, padding=1, groups=gradgrad_output.shape[1]
        )

        return grad_input, None, None

class BlurFunction(jt.Function):
    def execute(self, input, kernel, kernel_flip):
        self.saved_tensors = kernel, kernel_flip

        output = jt.nn.conv2d(input, kernel, padding=1, groups=input.shape[1])

        return output

    def grad(self, grad_output):
        kernel, kernel_flip = self.saved_tensors

        grad_input = BlurFunctionBackward().execute(grad_output, kernel, kernel_flip)

        return grad_input, None, None

blur = BlurFunction().apply

class Blur(jt.Module):
    def __init__(self, channel):
        weight = jt.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]], dtype='float32')
        weight = weight.reshape(1, 1, 3, 3)
        weight = weight / weight.sum()
        weight_flip = jt.flip(weight, [2, 3])

        self._weight = weight.repeat(channel, 1, 1, 1)
        self._weight_flip = weight_flip.repeat(channel, 1, 1, 1)

    def execute(self, input):
        return blur(input, self._weight, self._weight_flip)
        # return jt.nn.conv2d(input, self.weight, padding=1, groups=input.shape[1])

In [4]:
class FusedDownsample(jt.Module):
    def __init__(self, in_channel, out_channel, kernel_size, padding=0):
        self.weight = jt.randn(out_channel, in_channel, kernel_size, kernel_size)
        self.bias = jt.zeros(out_channel)

        fan_in = in_channel * kernel_size * kernel_size
        self.multiplier = sqrt(2 / fan_in)
        self.pad = padding

    def execute(self, input):
        weight = jt.nn.pad(self.weight * self.multiplier, [1, 1, 1, 1])
        weight = (
            weight[:, :, 1:, 1:]  +
            weight[:, :, :-1, 1:] +
            weight[:, :, 1:, :-1] +
            weight[:, :, :-1, :-1]
        ) / 4

        out = jt.nn.conv2d(input, weight, self.bias, stride=2, padding=self.pad)
        return out

class FusedUpsample(jt.Module):
    def __init__(self, in_channel, out_channel, kernel_size, padding=0):
        self.weight = jt.randn(in_channel, out_channel, kernel_size, kernel_size)
        self.bias = jt.zeros(out_channel)

        fan_in = in_channel * kernel_size * kernel_size
        self.multiplier = sqrt(2 / fan_in)
        self.pad = padding

    def execute(self, input):
        weight = jt.nn.pad(self.weight * self.multiplier, [1, 1, 1, 1])
        weight = (
            weight[:, :, 1:, 1:] +
            weight[:, :, :-1, 1:] +
            weight[:, :, 1:, :-1] +
            weight[:, :, :-1, :-1]
        ) / 4

        out = jt.nn.conv_transpose2d(input, weight, self.bias, stride=2, padding=self.pad)
        return out

## Discriminator

In [5]:
class ConvBlock(jt.Module):
    def __init__(
        self,
        in_channel,
        out_channel,
        kernel_size,
        padding,
        kernel_size2=None,
        padding2=None,
        downsample=False,
        fused=False
    ):
        pad1 = padding
        pad2 = padding
        if padding2 is not None:
            pad2 = padding2

        kernel1 = kernel_size
        kernel2 = kernel_size
        if kernel_size2 is not None:
            kernel2 = kernel_size2

        self.conv1 = jt.nn.Sequential(
            EqualConv2d(in_channel, out_channel, kernel1, padding=pad1),
            jt.nn.LeakyReLU(0.2),
        )

        if downsample:
            if fused:
                self.conv2 = jt.nn.Sequential(
                    Blur(out_channel),
                    FusedDownsample(out_channel, out_channel, kernel2, padding=pad2),
                    jt.nn.LeakyReLU(0.2),
                )
            else:
                self.conv2 = jt.nn.Sequential(
                    Blur(out_channel),
                    EqualConv2d(out_channel, out_channel, kernel2, padding=pad2),
                    jt.nn.AvgPool2d(2),
                    jt.nn.LeakyReLU(0.2),
                )
        else:
            self.conv2 = jt.nn.Sequential(
                EqualConv2d(out_channel, out_channel, kernel2, padding=pad2),
                jt.nn.LeakyReLU(0.2),
            )
    def execute(self, input):
        out = self.conv1(input)
        out = self.conv2(out)
        return out

In [10]:
class Discriminator(jt.Module):
    def __init__(self, fused=True, from_rgb_activate=False):
        self.progression = jt.nn.ModuleList(
            [
                #ConvBlock( 16,  32, 3, 1, downsample=True, fused=fused),  # 512
                #ConvBlock( 32,  64, 3, 1, downsample=True, fused=fused),  # 256
                #ConvBlock( 64, 128, 3, 1, downsample=True, fused=fused),  # 128
                ConvBlock(128, 256, 3, 1, downsample=True, fused=fused),  # 64
                ConvBlock(256, 512, 3, 1, downsample=True),  # 32
                ConvBlock(512, 512, 3, 1, downsample=True),  # 16
                ConvBlock(512, 512, 3, 1, downsample=True),  # 8
                ConvBlock(512, 512, 3, 1, downsample=True),  # 4
                ConvBlock(513, 512, 3, 1, 4, 0),
            ]
        )
        
        def make_from_rgb(out_channel):
            if from_rgb_activate:
                return jt.nn.Sequential(EqualConv2d(3, out_channel, 1), jt.nn.LeakyReLU(0.2))

            else:
                return EqualConv2d(3, out_channel, 1)

        self.from_rgb = jt.nn.ModuleList(
            [
                #make_from_rgb(16),
                #make_from_rgb(32),
                #make_from_rgb(64),
                make_from_rgb(128),
                make_from_rgb(256),
                make_from_rgb(512),
                make_from_rgb(512),
                make_from_rgb(512),
                make_from_rgb(512),
            ]
        )
        self.n_layer = len(self.progression)
        self.linear = EqualLinear(512, 1)
        
    def execute(self, input, step=0, alpha=-1):
        for i in range(step, -1, -1):
            index = self.n_layer - i - 1

            if i == step:
                out = self.from_rgb[index](input)

            if i == 0:
                out_std = np.std(out.data, axis=0)
                mean_std = jt.array(out_std.mean())
                mean_std = mean_std.expand((out.size(0), 1, 4, 4))
                out = jt.concat([out, mean_std], 1)

            out = self.progression[index](out)

            if i > 0:
                if i == step and 0 <= alpha < 1:
                    skip_rgb = jt.nn.avg_pool2d(input, 2)
                    skip_rgb = self.from_rgb[index + 1](skip_rgb)

                    out = (1 - alpha) * skip_rgb + alpha * out

        out = out.squeeze(2).squeeze(2)
        out = self.linear(out)
        return out

### Test Discriminator

In [11]:
netD = Discriminator()
nsteps = 6
alpha = 0
batch_size = 16
for step in range(nsteps):
    image_size = 2**(step+2)
    print('Testing image of [%02d x %02d]' % (image_size, image_size))
    image = jt.rand(batch_size, 3, image_size, image_size)
    pred = netD(image, step, alpha)
    assert(pred.shape == [batch_size, 1])

Testing image of [04 x 04]
Testing image of [08 x 08]
Testing image of [16 x 16]
Testing image of [32 x 32]
Testing image of [64 x 64]
Testing image of [128 x 128]



Compiling Operators(7/7) used: 2.37s eta:    0s 


## StyleGAN Generator

### StyledConvBlock

In [13]:
class ConstantInput(jt.Module):
    def __init__(self, channel, size=4):
        self.input = jt.randn(1, channel, size, size)

    def execute(self, input):
        batch = input.shape[0]
        out = self.input.repeat(batch, 1, 1, 1)

        return out

class NoiseInjection(jt.Module):
    def __init__(self, channel):
        self.weight = jt.zeros((1, channel, 1, 1))

    def execute(self, image, noise):
        return image + self.weight * noise

class AdaptiveInstanceNorm(jt.nn.Module):
    def __init__(self, in_channel, style_dim):
        self.norm = jt.nn.InstanceNorm2d(in_channel, affine=False)
        self.style = EqualLinear(style_dim, in_channel * 2)

        self.style.linear.bias.data[:in_channel] = 1
        self.style.linear.bias.data[in_channel:] = 0

    def execute(self, input, style):
        style = self.style(style).unsqueeze(2).unsqueeze(3)
        gamma, beta = style.chunk(2, 1)

        out = self.norm(input)
        out = gamma * out + beta

        return out 
    
class StyledConvBlock(jt.Module):
    def __init__(
        self,
        in_channel,
        out_channel,
        kernel_size=3,
        padding=1,
        style_dim=512,
        initial=False,
        upsample=False,
        fused=False,
    ):
        if initial:
            self.conv1 = ConstantInput(in_channel)
        else:
            if upsample:
                if fused:
                    self.conv1 = jt.nn.Sequential(
                        FusedUpsample(
                            in_channel, out_channel, kernel_size, padding=padding
                        ),
                        Blur(out_channel),
                    )
                else:
                    self.conv1 = jt.nn.Sequential(
                        jt.nn.Upsample(scale_factor=2, mode='nearest'),
                        EqualConv2d(
                            in_channel, out_channel, kernel_size, padding=padding
                        ),
                        Blur(out_channel),
                    )
            else:
                self.conv1 = EqualConv2d(
                    in_channel, out_channel, kernel_size, padding=padding
                )

        self.noise1 = equal_lr(NoiseInjection(out_channel))
        self.adain1 = AdaptiveInstanceNorm(out_channel, style_dim)
        self.lrelu1 = jt.nn.LeakyReLU(0.2)

        self.conv2  = EqualConv2d(out_channel, out_channel, kernel_size, padding=padding)
        self.noise2 = equal_lr(NoiseInjection(out_channel))
        self.adain2 = AdaptiveInstanceNorm(out_channel, style_dim)
        self.lrelu2 = jt.nn.LeakyReLU(0.2)
    
    def execute(self, input, style, noise):
        out = self.conv1(input)
        out = self.noise1(out, noise)
        out = self.lrelu1(out)
        out = self.adain1(out, style)

        out = self.conv2(out)
        out = self.noise2(out, noise)
        out = self.lrelu2(out)
        out = self.adain2(out, style)

        return out

### Syntheses Network

In [14]:
class PixelNorm(jt.Module):
    def __init__(self):
        pass

    def execute(self, input):
        return input / jt.sqrt(jt.mean(input ** 2, dim=1, keepdims=True) + 1e-8)

class Generator(jt.Module):
    def __init__(self, code_dim, fused=True):
        self.progression = jt.nn.ModuleList(
            [
                StyledConvBlock(512, 512, 3, 1, initial=True),   # 4
                StyledConvBlock(512, 512, 3, 1, upsample=True),  # 8
                StyledConvBlock(512, 512, 3, 1, upsample=True),  # 16
                StyledConvBlock(512, 512, 3, 1, upsample=True),  # 32
                StyledConvBlock(512, 256, 3, 1, upsample=True),  # 64
                #StyledConvBlock(256, 128, 3, 1, upsample=True, fused=fused),  # 128
                #StyledConvBlock(128,  64, 3, 1, upsample=True, fused=fused),  # 256
                #StyledConvBlock( 64,  32, 3, 1, upsample=True, fused=fused),  # 512
                #StyledConvBlock( 32,  16, 3, 1, upsample=True, fused=fused),  # 1024
            ]
        )

        self.to_rgb = jt.nn.ModuleList(
            [
                EqualConv2d(512, 3, 1),
                EqualConv2d(512, 3, 1),
                EqualConv2d(512, 3, 1),
                EqualConv2d(512, 3, 1),
                EqualConv2d(256, 3, 1),
                #EqualConv2d(128, 3, 1),
                #EqualConv2d(64, 3, 1),
                #EqualConv2d(32, 3, 1),
                #EqualConv2d(16, 3, 1),
            ]
        )

    def execute(self, style, noise, step=0, alpha=-1, mixing_range=(-1, -1)):
        out = noise[0]

        if len(style) < 2:
            inject_index = [len(self.progression) + 1]
        else:
            inject_index = sorted(random.sample(list(range(step)), len(style) - 1))

        crossover = 0

        for i, (conv, to_rgb) in enumerate(zip(self.progression, self.to_rgb)):
            if mixing_range == (-1, -1):
                if crossover < len(inject_index) and i > inject_index[crossover]:
                    crossover = min(crossover + 1, len(style))
                style_step = style[crossover]
            else:
                if mixing_range[0] <= i <= mixing_range[1]:
                    style_step = style[1]
                else:
                    style_step = style[0]

            if i > 0 and step > 0:
                out_prev = out
                
            out = conv(out, style_step, noise[i])

            if i == step:
                out = to_rgb(out)
                if i > 0 and 0 <= alpha < 1:
                    skip_rgb = self.to_rgb[i - 1](out_prev)
                    skip_rgb = jt.nn.interpolate(skip_rgb, scale_factor=2, mode='nearest')
                    out = (1 - alpha) * skip_rgb + alpha * out
                break

        return out

### Styled Generator

In [15]:
class StyledGenerator(jt.Module):
    def __init__(self, code_dim=512, n_mlp=8):
        self.generator = Generator(code_dim)

        layers = [PixelNorm()]
        for i in range(n_mlp):
            layers.append(EqualLinear(code_dim, code_dim))
            layers.append(jt.nn.LeakyReLU(0.2))

        self.style = jt.nn.Sequential(*layers)

    def execute(
        self,
        input,
        noise=None,
        step=0,
        alpha=-1,
        mean_style=None,
        style_weight=0,
        mixing_range=(-1, -1),
    ):
        styles = []
        if type(input) not in (list, tuple):
            input = [input]

        for i in input:
            styles.append(self.style(i))

        batch = input[0].shape[0]

        if noise is None:
            noise = []
            for i in range(step + 1):
                size = 4 * 2 ** i
                noise.append(jt.randn(batch, 1, size, size))

        if mean_style is not None:
            styles_norm = []
            
            for style in styles:
                styles_norm.append(mean_style + style_weight * (style - mean_style))

            styles = styles_norm

        return self.generator(styles, noise, step, alpha, mixing_range=mixing_range)

    def mean_style(self, input):
        style = self.style(input).mean(0, keepdims=True)

        return style

### Test Styled Generator

In [16]:
netG = StyledGenerator()
nsteps = 5
alpha = 0
batch_size = 16
code_dim = 512
z = jt.randn(batch_size, code_dim)

for step in range(nsteps):
    image_size = 2**(step+2)
    print('Testing image of [%02d x %02d]' % (image_size, image_size))
    fake_image = netG(z, step=step, alpha=alpha, style_weight=0.5)
    assert(fake_image.shape == [batch_size, 3, image_size, image_size])

Testing image of [04 x 04]
Testing image of [08 x 08]
Testing image of [16 x 16]
Testing image of [32 x 32]
Testing image of [64 x 64]
